In [0]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F

In [0]:

normalize = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,),(0.5,)),
                              ])

In [0]:
train_dataset = dsets.CIFAR100(root='./data', 
                            train=True, 
                            transform=normalize,
                            download=True)

test_dataset = dsets.CIFAR100(root='./data', 
                           train=False, 
                           transform=normalize)

Extracting ./data/cifar-100-python.tar.gz to ./data


In [0]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
class CNNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        #Conv 1
        self.cnn1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1 = nn.ReLU()
        
        #maxpool1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        #Conv 2
        self.cnn2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2 = nn.ReLU()

        #Conv 3
        self.cnn3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2)
        self.relu3 = nn.ReLU()

        
        #Maxpool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # 8 = (32/2)/2
        self.fc1 = nn.Linear(64*8*8,100)
        
    def forward(self,x):
        # x is of size (64,1,28,28)
        # reshape to (64,784)
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)

        out = self.cnn3(out)
        out = self.relu3(out)
        
        out = self.maxpool2(out)
        out = out.view(out.shape[0],-1)
        
        out = self.fc1(out)
        return out

In [0]:
model = CNNModel()

print(model)

CNNModel(
  (cnn1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (cnn3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu3): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=100, bias=True)
)


In [0]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=learning_rate)

In [0]:
epochs = 100
iter_n = 0
for e in range(epochs):
    print('Epoch {}'.format(e+1))
    running_loss = 0
    for i, (images,labels) in enumerate(train_loader):
        
        images = images.requires_grad_()
    
        
        optimizer.zero_grad()
        
        logits = model.forward(images)
        
        loss = criterion(logits,labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        iter_n += 1
        
        if iter_n%500 == 0:
            print('Iteration: {}.'.format(iter_n),end = ' ')
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
              
                    logit = model(images)
                    _, predicted = torch.max(logit.data, 1)
                    total += labels.size(0)

    
                    correct += (predicted == labels).sum()
                accuracy = 100 * correct / total
        
                print('Loss: {}. Accuracy: {}'.format(loss.item(), accuracy))

Epoch 1
Iteration: 500. Loss: 3.170485734939575. Accuracy: 26
Epoch 2
Iteration: 1000. Loss: 2.8403568267822266. Accuracy: 28
Epoch 3
Iteration: 1500. Loss: 2.619936227798462. Accuracy: 30
Epoch 4
Iteration: 2000. Loss: 2.533473491668701. Accuracy: 32
Epoch 5
Iteration: 2500. Loss: 2.4687652587890625. Accuracy: 32
Epoch 6
Iteration: 3000. Loss: 2.5582540035247803. Accuracy: 34
Epoch 7
Iteration: 3500. Loss: 2.3123624324798584. Accuracy: 34
Epoch 8
Iteration: 4000. Loss: 2.0468714237213135. Accuracy: 36
Epoch 9
Iteration: 4500. Loss: 1.9932681322097778. Accuracy: 37
Epoch 10
Iteration: 5000. Loss: 1.9884216785430908. Accuracy: 36
Epoch 11
Iteration: 5500. Loss: 2.260952949523926. Accuracy: 37
Epoch 12
Iteration: 6000. Loss: 1.8582762479782104. Accuracy: 37
Epoch 13
Iteration: 6500. Loss: 1.9969524145126343. Accuracy: 38
Epoch 14
Iteration: 7000. Loss: 1.570594310760498. Accuracy: 38
Epoch 15
Iteration: 7500. Loss: 1.8061401844024658. Accuracy: 37
Epoch 16
Iteration: 8000. Loss: 2.083354

KeyboardInterrupt: ignored